In [7]:
#this is test using velocity to reach the target not rudder


import gym
import numpy as np

class SailingEnv(gym.Env):
    def __init__(self):
        self.action_space = gym.spaces.Discrete(3)
        self.observation_space = gym.spaces.MultiDiscrete([10, 10])
        self.reset()

    def reset(self):
        self.current_pos = np.array([0, 0], dtype=np.float64)
        self.target = np.array([-5, 0], dtype=np.float64)
        self.velocity = np.array([1, 0], dtype=np.float64)
        return self._get_obs()

   # def step(self, action):
       # if action == 0:
           # self.velocity = np.array([1, 0], dtype=np.float64)
       # elif action == 1:
           # self.velocity = np.array([1, 1], dtype=np.float64)
       # elif action == 2:
         #   self.velocity = np.array([1, -1], dtype=np.float64)
    
    
    def step(self, action):
        # Define a vector of rudder angles for each possible action
        velocity_vectors = np.array([[1, 0], [1, 1], [1, -1]])
    
        # Set the rudder angle based on the action
        self.velocity = velocity_vectors[action]

        self.current_pos = np.float64(self.current_pos) + self.velocity * 0.1
        reward = -np.sqrt(np.sum(np.square(self.current_pos - self.target)))
        done = np.allclose(self.current_pos, self.target)
        info = {'current_pos': self.current_pos, 'reward': reward}
        return self._get_obs(), reward, done, info

    def _get_obs(self):
        return tuple(self.current_pos.astype(np.int64) + 5)


env = SailingEnv()

In [8]:
#q_table will have 10X10X4 cells
q_table = np.random.uniform(low=-2, high=0, size=([10,10] + [env.action_space.n]))

# Set learning parameters
LEARNING_RATE = 0.0001
DISCOUNT = 0.95
EPSILON=0.9

# Q-learning algorithm
num_episodes = 1000
alpha = 0.001
gamma = 0.9
epsilon = 0.1




# Set exploration parameters
#exploration_rate = 1.0
#max_exploration_rate = 1.0
#min_exploration_rate = 0.01
#exploration_decay_rate = 0.001


In [6]:
for i in range(num_episodes):
    done = False
    episode_reward = 0
    observation = env.reset()
    
    while not done:
        if np.random.random() < epsilon:
            # take a random action with probability epsilon
            action = env.action_space.sample()
        else:
            # choose the action with the highest Q-value
            action = np.argmax(q_table[observation[0], observation[1], :])
        
        next_observation, reward, done, _ = env.step(action)
        if (next_observation[0] < 0 or next_observation[0] > 9 or
            next_observation[1] < 0 or next_observation[1] > 9):
        # If next observation is out of bounds, set it to the previous observation
            next_observation = observation
    
        next_action = np.argmax(q_table[next_observation[0], next_observation[1], :])
        
        # update Q-value using Q-learning update rule
        q_table[observation[0], observation[1], action] += alpha * (reward + gamma * q_table[next_observation[0], next_observation[1], next_action] - q_table[observation[0], observation[1], action])
        observation = next_observation
        episode_reward += reward
        

KeyboardInterrupt: 